In [2]:
import arcpy
import os
from arcpy import env
from arcpy.sa import *

#### Define useful functions

In [3]:
# grab all rasters from a given folder folder or geodatabase
def get_rasters(workspace):
    # Get the current workspace
    previous_workspace = arcpy.env.workspace

    try:
        # Set the new workspace
        arcpy.env.workspace = workspace
        
        # Get a list of all feature classes in the workspace
        rasters = arcpy.ListRasters()

        # Return the list of feature classes
        return rasters

    finally:
        # Reset the workspace to the previous workspace
        arcpy.env.workspace = previous_workspace

In [4]:
# Function to perform reclassification
def discrete_reclassify(input_raster, remap_table):
    
    # sets scratchworkspace to scratch.gdb
    arcpy.env.scratchWorkspace = scratch_folder 

    # check if scratch folder exists, if not create it
    if not os.path.exists(arcpy.env.scratchWorkspace): 
        os.makedirs(arcpy.env.scratchWorkspace)   

    # Create the reclassified raster path
    reclassified_raster_name = os.path.splitext(input_raster)[0] + "_reclass.tif"

    reclassified_raster_path = os.path.join(scratch_folder, reclassified_raster_name)

    # Get the remap table path
    remap_table_path = os.path.join(remap_table_gdb, remap_table)

    # Parse the geodatabase table and create a remap object
    remap_value = []
    with arcpy.da.SearchCursor(remap_table_path, ["OldValue", "NewValue"]) as cursor:
        for row in cursor:
            remap_value.append(tuple(row))

    # Create a RemapRange object
    remap = arcpy.sa.RemapValue(remap_value)


    # Reclassify raster
    reclassified_raster = Reclassify(in_raster = input_raster, reclass_field = "Value", remap = remap, missing_values = "DATA")
    reclassified_raster.save(reclassified_raster_path)

In [5]:
def continuous_reclassify(input_raster, remap_table):
    # sets scratchworkspace to scratch.gdb
    arcpy.env.scratchWorkspace = scratch_folder 

    # check if scratch folder exists, if not create it
    if not os.path.exists(arcpy.env.scratchWorkspace): 
        os.makedirs(arcpy.env.scratchWorkspace)
    
    # Create the reclassified raster path
    reclassified_raster_name = os.path.splitext(input_raster)[0] + "_reclass.tif"

    reclassified_raster_path = os.path.join(scratch_folder, reclassified_raster_name)

    # Get the remap table path
    remap_table_path = os.path.join(remap_table_gdb, remap_table)

    # Parse the geodatabase table and create a remap object
    remap_range = []
    with arcpy.da.SearchCursor(remap_table_path, ["FromValue", "ToValue", "OutputValue"]) as cursor:
        for row in cursor:
            remap_range.append(row)

    # Create a RemapRange object
    remap = arcpy.sa.RemapRange(remap_range)

    # Reclassify raster
    reclassified_raster = Reclassify(in_raster = input_raster, reclass_field = "VALUE", remap = remap, missing_values = "DATA")
    reclassified_raster.save(reclassified_raster_path)

#### Set input and output paths, set environmental variables for raster analysis

In [6]:
# set input and output folder, create output file geodatabase
mainInputFolder = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data" # enter path to input folder
mainOutputFolder = "C:\\Users\\Zachary\\ASSET\\Transmission\\analysis\\data\\" # enter path to output folder

gdbFileName = "leastCostPathResults.gdb" # enter name of output geodatabase
gdbPath = os.path.join(mainOutputFolder, gdbFileName)
if not arcpy.Exists(gdbPath):
    arcpy.CreateFileGDB_management(mainOutputFolder, gdbFileName)
else:
    print(f"Geodatabase '{gdbFileName}' already exists at '{mainOutputFolder}'.")

Geodatabase 'leastCostPathResults.gdb' already exists at 'C:\Users\Zachary\ASSET\Transmission\analysis\data\'.


In [7]:
# set scratchworkspace to output workspace
scratch_folder = os.path.join(mainOutputFolder, "scratch")
if not arcpy.Exists(scratch_folder):
    os.makedirs(scratch_folder)
    print("Scratch folder created successfully.")
else:
    print(f"'{scratch_folder}' already exists at '{mainOutputFolder}'.")

'C:\Users\Zachary\ASSET\Transmission\analysis\data\scratch' already exists at 'C:\Users\Zachary\ASSET\Transmission\analysis\data\'.


In [8]:
# set path to remap tables
remap_table_gdb = os.path.join(mainInputFolder, "remapTables.gdb") # enter path to remap table geodatabase

In [32]:
# set environments
scratch_folder = os.path.join(mainOutputFolder, "scratch") # creates scratch geodatabase in output folder
raster_workspace = os.path.join(mainInputFolder, "inputRasters")


arcpy.env.scratchWorkspace = scratch_folder
arcpy.env.workspace = raster_workspace
arcpy.env.overwriteOutput = True

In [11]:
# set input paths:
slope_raster = os.path.join(mainInputFolder, "inputRasters\\slope.tif") # enter path to dem raster
nlcd_reclass = os.path.join(scratch_folder, "nlcd_cat_reclass.tif") # enter path to reclassified landcover raster

In [12]:
# set environmental raster settings to the DEM raster, don't forget to resample cell sizes to 250m 
arcpy.env.snapRaster = slope_raster
arcpy.env.extent = slope_raster

#### Reclassify Input Rasters

In [39]:

cat_raster_list = [raster for raster in arcpy.ListRasters("*cat*")]
cont_raster_list = [raster for raster in arcpy.ListRasters() if "_cat" not in raster]
cat_raster_list

['exclusion_cat.tif', 'whp2020_cat.tif']

In [15]:
for raster in cat_raster_list:

    # set remap table name
    remap_table = os.path.splitext(raster)[0] + "_remap_table"

    discrete_reclassify(raster, remap_table)
        
print("categorical rasters reclassified successfully")
    

categorical rasters reclassified successfully


In [16]:
for raster in cont_raster_list:

    # set remap table name
    remap_table = os.path.splitext(raster)[0] + "_remap_table"

    continuous_reclassify(raster, remap_table)
        
print("continuous rasters reclassified successfully")

continuous rasters reclassified successfully


#### Create Cost Surface

In [62]:
arcpy.env.workspace = scratch_folder
arcpy.env.overwriteOutput = True
arcpy.env.snapRaster = slope_raster
arcpy.env.extent = slope_raster

raster_reclass_list = [Raster(raster) for raster in arcpy.ListRasters()]
raster_reclass_list

[exclusion_cat_reclass.tif,
 landscan_reclass.tif,
 nlcd_cat_reclass.tif,
 slope_reclass.tif,
 whp2020_cat_reclass.tif]

In [63]:
# Build the raster calculator expression to calculate the final output raster
expression = '*'.join([f'"{raster}"*.001' for raster in raster_reclass_list])
expression

'"exclusion_cat_reclass.tif"*.001*"landscan_reclass.tif"*.001*"nlcd_cat_reclass.tif"*.001*"slope_reclass.tif"*.001*"whp2020_cat_reclass.tif"*.001'

In [64]:
# set output path for raster calculator expression
output_path = os.path.join(mainInputFolder, "CostRaster\\costSurface.tif")
arcpy.gp.RasterCalculator_sa(expression, output_path)

<geoprocessing server result object at 0x176a8a133f0>

#### Create cost surface sans Fire Risk 

In [65]:
raster_reclass_no_fire = [Raster(raster) for raster in arcpy.ListRasters() if "whp" not in raster]
raster_reclass_no_fire

[exclusion_cat_reclass.tif,
 landscan_reclass.tif,
 nlcd_cat_reclass.tif,
 slope_reclass.tif]

In [66]:
# Build the raster calculator expression to calculate the final output raster
expression = '*'.join([f'"{raster}"*.001' for raster in raster_reclass_no_fire])
expression

'"exclusion_cat_reclass.tif"*.001*"landscan_reclass.tif"*.001*"nlcd_cat_reclass.tif"*.001*"slope_reclass.tif"*.001'

In [67]:
# set output path for raster calculator expression
output_path = os.path.join(mainInputFolder, "CostRaster\\costSurfaceNoFire.tif")
arcpy.gp.RasterCalculator_sa(expression, output_path)

<geoprocessing server result object at 0x176af4078d0>